In [11]:
import pandas as pd
from surprise import Dataset, Reader
import io
import requests

# MovieLens 100K 데이터셋 URL (u.data 파일)
url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.data"

# 데이터셋 다운로드
response = requests.get(url).content

# BytesIO 객체로 변환
data_io = io.BytesIO(response)

# DataFrame으로 읽기
data_df = pd.read_csv(data_io, sep="\t", names=["userID", "itemID", "rating", "timestamp"])

# timestamp 컬럼은 필요 없으므로 제거합니다.
data_df.drop("timestamp", axis=1, inplace=True)

# Reader 객체 생성: MovieLens 100k의 등급 범위는 1부터 5입니다.
reader = Reader(rating_scale=(1, 5))

# DataFrame 로드
data = Dataset.load_from_df(data_df[["userID", "itemID", "rating"]], reader)

In [12]:
from surprise import SVD
from surprise.model_selection import cross_validate

# SVD 알고리즘 선택 (Singular Value Decomposition, 특이값 분해)
algo = SVD()

# 5-fold 교차 검증 실행 및 결과 출력
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9361  0.9460  0.9280  0.9383  0.9364  0.9370  0.0057  
MAE (testset)     0.7377  0.7430  0.7316  0.7382  0.7397  0.7380  0.0037  
Fit time          0.70    0.67    0.68    0.69    0.67    0.68    0.01    
Test time         0.06    0.06    0.11    0.06    0.06    0.07    0.02    


{'test_rmse': array([0.93613127, 0.94597308, 0.92803562, 0.93830963, 0.93643896]),
 'test_mae': array([0.73772941, 0.74295489, 0.73155296, 0.73820558, 0.73971802]),
 'fit_time': (0.6959061622619629,
  0.6680014133453369,
  0.6794655323028564,
  0.6939043998718262,
  0.6689701080322266),
 'test_time': (0.061036109924316406,
  0.058995723724365234,
  0.11099815368652344,
  0.05899953842163086,
  0.06002473831176758)}

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# 가정: 사용자 ID와 아이템 ID가 각각 0부터 시작하는 정수라고 가정하였습니다.
n_users = 1000  # 전체 사용자 수
n_items = 500  # 전체 아이템 수

# 임베딩 차원 설정
embedding_dim = 50

# 입력 데이터 (사용자 ID와 아이템 ID)
user_input = torch.randint(high=n_users, size=(1024,))
item_input = torch.randint(high=n_items, size=(1024,))
context_input = torch.rand(
    1024,
)  # 예: 현재 시간을 나타내는 실수값

# 출력 데이터 (사용자가 해당 아이템에 대해 클릭을 했는지 여부)
output_data = torch.randint(low=0, high=2, size=(1024,))


class Recommender(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        self.lstm_layer = nn.LSTM(embedding_dim * 2, 128, batch_first=True)
        self.fc_layer = nn.Linear(128, 1)

    def forward(self, x):
        user_embedded = self.user_embedding(x[:, 0])
        item_embedded = self.item_embedding(x[:, 1])

        x = torch.cat((user_embedded, item_embedded), dim=1).unsqueeze(1)

        output, _ = self.lstm_layer(x)

        return self.fc_layer(output[:, -1, :])


model = Recommender()

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_data = torch.stack((user_input, item_input), dim=-1)

for epoch in range(100):
    optimizer.zero_grad()

    output = model(input_data)

    loss = criterion(output.squeeze(), output_data.float())

    loss.backward()

    optimizer.step()

In [5]:
# 가정: test_user_input, test_item_input, test_output_data는 테스트 데이터셋입니다.
test_user_input = torch.randint(high=n_users, size=(256,))
test_item_input = torch.randint(high=n_items, size=(256,))
test_output_data = torch.randint(low=0, high=2, size=(256,))


input_data_test = torch.stack((test_user_input, test_item_input), dim=-1)

model.eval()  # 모델을 평가 모드로 설정

with torch.no_grad():  # 그래디언트 계산 비활성화
    predictions = model(input_data_test)
    loss = criterion(predictions.squeeze(), test_output_data.float())

print(f"Test Loss: {loss.item()}")

Test Loss: 1.256568193435669
